In [ ]:
from sqlalchemy import create_engine


engine = create_engine("sqlite:///example.db", echo=False)

In [ ]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped, mapped_column


class Base(DeclarativeBase):
    pass


class ToDo(Base):
    __tablename__ = "todos"

    id: Mapped[int] = mapped_column(primary_key=True)
    title: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[str] = mapped_column(nullable=True)

In [ ]:
# Create the table in the database
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import Session


with Session(engine) as session:
    todo = ToDo(title="Buy groceries", description="Buy groceries")
    session.add(todo)
    session.commit()

In [ ]:
from sqlalchemy import select


with Session(engine) as session:
    statement = select(ToDo)
    result = session.execute(statement=statement)
    todos = result.scalars().all()

for todo in todos:
    for key, value in todo.__dict__.items():
        print(key, value)


In [ ]:
todos = [
    ToDo(title="Buy groceries", description="Buy groceries"),
    ToDo(title="Get a haircut", description="Get a haircut"),
    ToDo(title="Buy a new phone", description="Buy a new phone"),
    ToDo(title="Buy a new laptop", description="Buy a new laptop"),
    ToDo(title="Buy a new car", description="Buy a new car"),
    ToDo(title="Buy a new house", description="Buy a new house"),
    ToDo(title="Buy a new boat", description="Buy a new boat"),
    ToDo(title="Buy a new plane", description="Buy a new plane"),
    ToDo(title="Buy a new island", description="Buy a new island"),
    ToDo(title="Buy a new moon", description="Buy a new moon"),
    ToDo(title="Buy a new sun", description="Buy a new sun"),
]

with Session(engine) as session:
    session.add_all(todos)
    session.commit()

In [ ]:
with Session(engine) as session:
    statement = select(ToDo).filter_by(title="Buy a new phone")
    todo = session.execute(statement=statement).scalar_one()
    todo.title = "Buy a new charger"
    todo.description = "Buy a new charger"
    session.commit()

In [ ]:
with Session(engine) as session:
    statement = select(ToDo).filter_by(title="Buy a new charger")
    todo = session.execute(statement=statement).scalar_one()
    session.delete(todo)
    session.commit()

In [ ]:
# Core foundations of SQLAlchemy ORM
from sqlalchemy import create_engine


engine = create_engine("sqlite:///example.db", echo=False)

from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()

todos_table = Table(
    "todos", 
    metadata, 
    Column("id", Integer, primary_key=True),
    Column("title", String, nullable=True),
    Column("description", String, nullable=True)
)

metadata.create_all(engine)

In [ ]:
# Inserting Data
from sqlalchemy import insert

statement = insert(todos_table).values(title="Walk the dog", description="Walk the dog in the park")

with engine.connect() as connection:
    connection.execute(statement)
    connection.commit()


# Selecting Data
from sqlalchemy import select


statement = select(todos_table)

with engine.connect() as connection:
    result = connection.execute(statement)
    todos = result.fetchall()
    for todo in todos:
        print(todo)

ORM Basics

In [ ]:
import datetime
from sqlalchemy.orm import declarative_base

Base = declarative_base()


from sqlalchemy import Column, Integer, String, DateTime, Boolean

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=True)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.datetime.now)


Base.metadata.create_all(engine)

In [ ]:
# Sessions (VERY IMPORTANT)
from sqlalchemy.orm import sessionmaker

SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

In [ ]:
users = [
    User(name="Alice"),
    User(name="Bob"),
    User(name="Charlie"),
    User(name="Dave"),
    User(name="Eve"),
]

for user in users:
    session.add(user)
    session.commit()

session.add_all(users)
session.commit()

users = session.query(User).all()

for user in users:
    print(user.id, user.name, user.is_active, user.created_at)

In [ ]:
users = session.query(User).filter(User.name == "Alice").all()
for user in users:
    print(user.id, user.name, user.is_active, user.created_at)

In [ ]:
from typing import List, Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey


class Category(Base):
    __tablename__ = "categories"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[Optional[str]] = mapped_column(nullable=True)
    products: Mapped[List["Product"]] = relationship(back_populates="category")

    def __repr__(self) -> str:
        return f"Category(id={self.id!r}, name={self.name!r}, description={self.description!r})"


class Product(Base):
    __tablename__ = "products"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(nullable=True)
    description: Mapped[str] = mapped_column(nullable=True)
    category_id: Mapped[int] = mapped_column(ForeignKey("categories.id"))

    category: Mapped[Category] = relationship(back_populates="products")

    def _repr__(self) -> str:
        return f"Product(id={self.id!r}, name={self.name!r}, description={self.description!r}, category_id={self.category_id!r})"

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
categories = [
    Category(name="Electronics", description="Electronic items"),
    Category(name="Clothing", description="Clothing items"),
    Category(name="Books", description="Books and literature"),
]

for category in categories:
    session.add(category)
    session.commit()

products = [
    Product(name="Laptop", description="A personal computer", category_id=1),
    Product(name="Smartphone", description="A mobile phone", category_id=1),
    Product(name="T-Shirt", description="A cotton t-shirt", category_id=2),
    Product(name="Jeans", description="Denim jeans", category_id=2),
    Product(name="Novel", description="A fiction book", category_id=3),
    Product(name="Biography", description="A non-fiction book", category_id=3),
]

session.add_all(products)
session.commit()

In [ ]:
categories = session.query(Category).all()
for category in categories:
    print(category)

products = session.query(Product).all()
for product in products:
    for key, value in product.__dict__.items():
        print(key, value)

In [ ]:
from os import name


def print_products_details(products):
    for product in products:
        for key, value in product.__dict__.items():
            print(key, value)

def print_product_details(product):
    if product:
        for key, value in product.__dict__.items():
            print(key, value)

products = session.query(Product).filter(Product.name == "Laptop").all()
print_products_details(products)

products = session.query(Product).filter(Product.name.icontains("Laptop")).all()
print_products_details(products)

products = session.query(Product).filter(Product.category.has(name="Electronics")).all()
print_products_details(products)

products = session.query(Product).filter(Product.name.ilike("%phone%"), Product.category.has(name="Electronics")).all()
print_products_details(products)

product = session.query(Product).filter(Product.name.ilike("%top%"), Product.category.has(name="Electronics")).first()
print_product_details(product)

product = session.query(Product).filter_by(id=1, name="Laptop").first()
print_product_details(product)

In [ ]:
product = session.query(Product).filter(Product.id == 7).first()
product.name = "Laptop"
print(product)
session.commit()
product = session.query(Product).filter(Product.id == 7).first()
print_product_details(product)

In [ ]:
product = session.query(Product).filter(Product.id == 4).first()
session.delete(product)
session.commit()

products = session.query(Product).all()
for product in products:
    for key, value in product.__dict__.items():
        if key == "name":
            print(value)

Ordering & Pagination

In [ ]:
products = session.query(Product).order_by(Product.name.asc()).limit(2).offset(2)
print_products_details(products)

Aggregations

In [ ]:
count = session.query(Product.category_id).distinct().count()
print("Distinct category count:", count)

In [ ]:
from sqlalchemy import func



count = session.query(func.count(Product.category_id).distinct()).scalar()
print("Distinct category count:", count)

_sum = session.query(func.sum(Product.category_id)).scalar()
print("Sum of category IDs:", _sum)

avg = session.query(func.avg(Product.category_id)).scalar()
print("Average of category IDs:", avg)

_max = session.query(func.max(Product.category_id)).scalar()
print("Maximum of category IDs:", _max)

_min = session.query(func.min(Product.category_id)).scalar()
print("Minimum of category IDs:", _min)

count, avg, _sum, _max, _min = session.query(
    func.count(Product.category_id), 
    func.avg(Product.category_id),
    func.sum(Product.category_id),
    func.max(Product.category_id),
    func.min(Product.category_id),
).one()

print("Distinct category count:", count)
print("Average of category IDs:", avg)
print("Sum of category IDs:", _sum)
print("Maximum of category IDs:", _max)
print("Minimum of category IDs:", _min)

In [ ]:
products = session.query(Product).all()
for product in products:
    print(product.category.name)

In [ ]:
categories = session.query(Category).join(Category.products).distinct().all()

for category in categories:
    print(category.name, ": ", ", ".join([str(product.name) for product in category.products]))



In [ ]:
categories = session.query(Category).join(Product)
for category in categories:
    print([product.__dict__ for product in category.products])
    for key, value in category.__dict__.items():
        print(key, value)